<a href="https://colab.research.google.com/github/pranavsrinivasa/SDXL-SAM-HEADSHOT-CREATOR/blob/main/SDXL-SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers
!pip install transformers

In [2]:
from diffusers import StableDiffusionInpaintPipeline
import torch

model_id = 'stabilityai/stable-diffusion-2-inpainting'
sd_pipeline = StableDiffusionInpaintPipeline.from_pretrained(model_id,torch_dtype = torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
!pip install gradio==3.48.0

In [4]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

  Preparing metadata (setup.py) ... done


In [5]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [6]:
import os

CHECKPOINT_PATH = os.path.join("{HOME}", "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))
MODEL_TYPE = "vit_h"

{HOME}/weights/sam_vit_h_4b8939.pth ; exist: True


In [7]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to("cuda")

In [8]:
predictor = SamPredictor(sam)

In [17]:
import gradio as gr
import numpy as np
from PIL import Image

selected_pixels = []
isInvert = 0

with gr.Blocks() as genaieg:
  selected_pixels = []
  isInvert = 0
  with gr.Row():
    input_img = gr.Image(label = 'Input')
    mask_img = gr.Image(label = "Mask")
  with gr.Row():
    output_img = gr.Image(label = "Ouput")

  def invertmask():
    global isInvert
    isInvert = not(isInvert)

  with gr.Row():
    prompt_text = gr.Textbox(line = 1,label = 'Prompt')
    submit = gr.Button('Submit')
    radio = gr.Radio(['Invert Mask'])
    radio.select(fn = invertmask)

  def generate_mask(image, evt: gr.SelectData):
    selected_pixels.append(evt.index)
    predictor.set_image(image)
    input_points = np.array(selected_pixels)
    input_label = np.ones(input_points.shape[0])
    mask, _, _ = predictor.predict(
        point_coords = input_points,
        point_labels = input_label,
        multimask_output = False
    )
    if isInvert:
      mask = np.logical_not(mask)
    mask = Image.fromarray(mask[0,:,:])
    return mask


  def inpaint(img, mask, prompt):
    img = Image.fromarray(img)
    mask = Image.fromarray(mask)
    img = img.resize((512,512))
    mask = mask.resize((512,512))
    negative_prompts = """
    duplicate,low quality, lowest quality, bad shape,bad anatomy,
    bad proportions, lowres,error,watermark,username,artistname,
    signature,text,jpeg artifacts,blurry,more than one person,simple background
    """
    prompt_text = "Realistic professinal Headshot of a man for a profile pic" + prompt
    output = sd_pipeline(prompt = prompt_text,
                         image = img,
                         negative_prompt = negative_prompts,
                         mask_image = mask).images[0]

    return output


  input_img.select(generate_mask, [input_img],[mask_img])
  submit.click(inpaint,
               inputs=[input_img,mask_img,prompt_text],
               outputs = [output_img])

if __name__ == '__main__':
  genaieg.launch(debug = True)

<ipython-input-17-aa308a6eb3d0>:22: GradioUnusedKwargWarning: You have unused kwarg parameters in Textbox, please remove them: {'line': 1}
  prompt_text = gr.Textbox(line = 1,label = 'Prompt')


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6e8ca624eb9397d7cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6e8ca624eb9397d7cf.gradio.live
